---
title: "Preprocessing data with the skrub `Cleaner`"
format:
    revealjs:
        slide-number: true
        toc: true
        code-fold: false
        code-tools: true

---

## Introduction
In this chapter, we will show how we can quickly pre-process and sanitize 
data using skrub's `Cleaner`. 

We first load the `wine` datasets from OpenML. 

In [1]:
from sklearn.datasets import fetch_openml
from skrub import TableReport
import pandas as pd

data = fetch_openml(data_id=42074)
wine = data.data

ImportError: cannot import name 'TableReport' from 'skrub' (unknown location)

We can explore it using the `TableReport`:

In [ ]:
TableReport(wine)

We can notice that there are a few columns that contain a sizable amount of 
missing values ("region_2" and "designation"). If we want to remove these columns
programmatically using pandas, we have to do something like this: 

In [ ]:
wine.loc[:, wine.isnull().mean() <= 0.3]

It may also be beneficial to convert numerical features to `float32`, to reduce
the computational cost: 

In [ ]:
wine.astype({col: "float32" for col in wine.select_dtypes(include="number").columns})

These operations are quite common in most cases (although the parameters and 
requirements may vary by project), so writing the code that addresses them may 
become repetitive.

A simpler way of dealing with this preliminary preparation is to use the skrub
`Cleaner`. 

## Using the skrub `Cleaner`
The `Cleaner` is intended to be a first step in preparing tabular data for 
analysis or modeling, and can handle a variety of common data cleaning tasks
automatically. It is designed to work out-of-the-box with minimal configuration,
although it is also possible to customize its behavior if needed.


Given a dataframe, the `Cleaner` applies a sequence of transformers to each column:

Consider this example dataframe:

In [ ]:
df = pd.DataFrame(
    {
        "numerical_1": [1, 2, 3, 4, 5],
        "numerical_2": [10.5, 20.3, None, 40.1, 50.2],
        "string_column": ["apple", "?", "banana", "cherry", "?"],
        "datetime_column": [
            "03 Jan 2020",
            "04 Jan 2020",
            "05 Jan 2020",
            "06 Jan 2020",
            "07 Jan 2020",
        ],
        "all_none": [None, None, None, None, None],
    }
)
df

This dataframe has mixed type columns, with some of the missing values denoted
as `None` and some `"?"`. The datetime column has a non-standard format and has
been parsed as a string column. Finally, one of the columns is completely empty. 

In [ ]:
df.info()

By default, the `Cleaner` applies various transformations that can sanitize many
common use cases:

In [ ]:
from skrub import Cleaner
df_clean = Cleaner().fit_transform(df)
df_clean

We can see that the cleaned version of the dataframe is now marking missing values
correctly, and that the datetime column has been parsed accordingly:

In [ ]:
df_clean.info()

### Cleaning steps performed by the `Cleaner` 
In more detail, the `Cleaner` executes the following steps in order: 

1. It replaces common strings used to represent missing values (e.g., `NULL`, `?`)
with NA markers. 
2. It uses the `DropUninformative` transformer to decide whether a column is 
"uninformative", that is, it is not likely to bring information useful to train
a ML model. For example, empty columns are uninformative. 
3. It tries to parse datetime columns using common formats, or a user-provided
`datetime_format`. 
4. It processes categorical columns to ensure consistent typing depending on the 
dataframe library in use. 
5. It converts columns to string, unless they have a data type that carries more 
information, such as numerical, datetime, and categorial columns.
6. Finally, it can convert numerical columns to `np.float32` dtype. This ensures 
a consistent representation of numbers and missing values, and helps reducing 
the memory footprint. 

We can look back at the "wine" dataframe and clean it with a suitably configured
`Cleaner`:


In [ ]:
cleaner = Cleaner(drop_null_fraction=0.3, numeric_dtype="float32")

cleaner.fit_transform(wine)

## Under the hood: `DropUninformative`
When the `Cleaner` is fitted on a dataframe, it checks whether the dataframe includes
uninformative columns, that is columns that do not bring useful information for 
training a ML model, and should therefore be dropped.

This is done by the `DropUninformative` transformer, which is a standalone transformer
that the `Cleaner` leverages to sanitize data. 
`DropUninformative` marks a columns as "uninformative" if it satisfies one of these 
conditions:

- The fraction of missing values is larger than the threshold provided by the user
with `drop_null_fraction`. 
    - By default, this threshold is 1.0, i.e., only columns
    that contain only missing values are dropped. 
    - Setting the threshold to `None` will disable this check and therefore retain
    empty columns. 
- It contains only one value, and no missing values. 
    - This is controlled by the `drop_if_constant` flag, which is `False` by 
    default. 
- All values in the column are distinct. 
    - This may be the case if the column contains
    UIDs, but it can also happen when the column contains text. 
    - This check is off by default and can be turned on by setting 
    `drop_if_unique` to `True`. 


# Exercise: clean a dataframe using the `Cleaner` 
Load the given dataframe. 

In [ ]:
import pandas as pd
df = pd.read_csv("../data/synthetic_data.csv")

Use the `TableReport` to answer the following questions: 

- Are there constant columns? 
- Are there datetime columns? If so, were they parsed correctly? 
- What is the dtype of the numerical features? 

In [ ]:
from skrub import TableReport
TableReport(df)

Then, use the `Cleaner` to sanitize the data so that:
- Constant columns are removed
- Datetimes are parsed properly (hint: use `"%d-%b-%Y"` as the datetime format)
- All columns with more than 50% missing values are removed
- Numerical features are converted to `float32`

In [ ]:
from skrub import Cleaner

# Write your answer here
# 
# 
# 
# 
# 
# 
# 
# 

In [ ]:
# solution
from skrub import Cleaner

cleaner = Cleaner(
    drop_if_constant=True,
    drop_null_fraction=0.5,
    numeric_dtype="float32",
    datetime_format="%d-%b-%Y",
)

# Apply the cleaner
df_cleaned = cleaner.fit_transform(df)

# Display the cleaned dataframe
TableReport(df_cleaned)

We can inspect which columns were dropped and what transformations were applied:

In [ ]:
print(f"Original shape: {df.shape}")
print(f"Cleaned shape: {df_cleaned.shape}")
print(
    f"\nColumns dropped: {[col for col in df.columns if col not in cleaner.all_outputs_]}"
)